In [1]:
import pandas as pd
import os
from tqdm import tqdm
from IPython.display import clear_output

# os.sys.path.append("/data/kai/forecasting/multimodal/financial/src")
# from utils import download_raw_texts_from_urls, save_text_to, load_text_from 
from glob import glob
from tqdm import trange  
import matplotlib.pyplot as plt
import numpy as np


In [6]:
paths = sorted(glob("/data/kai/forecasting/data/raw_v0.2/*.csv"))

In [8]:
blocked_words = [
    "thestreet.comPlease enable JS and disable any ad blocker",
    "wsj.comPlease enable JS and disable any ad blocker",
    "Access Denied Access Denied You don't have permission to access",
    "Access to this page has been denied",
    "Sorry! Temporarily Unavailable Sorry, this page is temporarily unavailable for technical reasons."
]


In [9]:
import re
import pandas as pd

blocked_words = [
    "thestreet.comPlease enable JS and disable any ad blocker",
    "wsj.comPlease enable JS and disable any ad blocker",
    "Access Denied Access Denied You don't have permission to access",
    "Access to this page has been denied",
    "Sorry! Temporarily Unavailable Sorry, this page is temporarily unavailable for technical reasons."
]

# Create a regex pattern from the blocked words
blocked_pattern = '|'.join([re.escape(word.lower()) for word in blocked_words])

total_count = 0
blocked_count = 0

for path in paths:
    df = pd.read_csv(path)
    df = df.fillna('')
    total_count += df.shape[0]

    # Count rows containing any of the blocked words or empty text
    blocked_count += df[
        df['text'].str.lower().str.contains(blocked_pattern) | (df['text'].str == '')
    ].shape[0]

print(f"Total count: {total_count}")
print(f"Blocked count: {blocked_count}")


Total count: 196303
Blocked count: 7371


In [43]:
blocked_count, total_count, blocked_count / total_count

(8238, 183964, 0.04478050053271292)

In [ ]:
data_dir = "/data/kai/forecasting/summary"

summary_paths = glob(data_dir + "/*/*_summary.txt")

In [ ]:
blank_idxs = []
for i, path in enumerate(summary_paths):
    summary = load_text_from(path)
    summary = '\n'.join(summary)
    if summary == '':
        blank_idxs.append(i)

In [ ]:
len(summary_paths)

In [ ]:
len(blank_idxs)

In [ ]:
[s for i, s in enumerate(summary_paths) if i in blank_idxs]

In [ ]:
directory_path = '/data/kai/forecasting/raw_urls'
file_names = os.listdir(directory_path)

ticker = "aapl"
df = pd.read_csv(directory_path + f'/{ticker}_text.csv')[::-1]

data_dir = "/data/kai/forecasting/summary"

for idx, date_str in enumerate(df["timestamp"].unique()):
    if os.path.exists(f"{data_dir}/{ticker}_{date_str}_final_summary.txt"):
        continue
    break

In [ ]:
# Plot 
temp_df = df["timestamp"].value_counts().reset_index()
temp_df = temp_df.sort_values(by="timestamp")

plt.figure(figsize=(14, 7))
plt.plot(temp_df['timestamp'], temp_df['count'], marker='o')

# Add titles and labels
plt.title('Number of URLs per Day')
plt.xlabel('Date')
plt.ylabel('Number of URLs')
plt.xticks(temp_df['timestamp'][::50], rotation=45)
plt.savefig("/data/kai/forecasting/eda_plots/AAPL_urls_per_day.png")
plt.show()


# Plot 
temp_df = df["timestamp"].value_counts().reset_index()
temp_df = temp_df.sort_values(by="timestamp")[30:]

plt.figure(figsize=(14, 7))
plt.plot(temp_df['timestamp'], temp_df['count'], marker='o')

# Add titles and labels
plt.title('Number of URLs per Day without Outliers')
plt.xlabel('Date')
plt.ylabel('Number of URLs')
plt.xticks(temp_df['timestamp'][::50], rotation=45)
plt.savefig("/data/kai/forecasting/eda_plots/AAPL_urls_per_day_without_outliers.png")
plt.show()

In [ ]:
missing_counts = []
all_counts = []
for ticker in file_names:
    df = pd.read_csv(directory_path + "/" + ticker)[::-1]
    timestamps = pd.to_datetime(df["timestamp"].unique())
    
    # dates which are not in the sequence 
    # are returned
    missing_dates = pd.date_range(start=df["timestamp"].iloc[-1], end=df["timestamp"].iloc[0]).difference(timestamps)
    missing_counts.append(len(missing_dates))
    all_counts.append(len(timestamps))

    # if len(missing_dates) < len(timestamps):
    #     break
    # print(ticker, "Missing dates:", missing_dates.shape[0], unique_dates.shape[0], df.shape[0], df['timestamp'].iloc[-1], df['timestamp'].iloc[0])

In [ ]:
np.array(missing_counts) / (np.array(all_counts) + np.array(missing_counts))

In [ ]:

# Calculate the ratio of missing to all days
ratios = np.array(missing_counts) / (np.array(all_counts) + np.array(missing_counts))

# Create a list of tuples (file_name, missing_count, all_count, ratio)
data = list(zip(file_names, missing_counts, all_counts, ratios))

# Sort the data by all_counts in ascending order
sorted_data = sorted(data, key=lambda x: x[2])

# Unpack the sorted data
sorted_file_names, sorted_missing_counts, sorted_all_counts, sorted_ratios = zip(*sorted_data)

# Generate labels
labels = [f.replace('_text.csv', '') for f in sorted_file_names]

# Set the width of the bars
bar_width = 0.35

# Set the position of the bars on the x-axis
r1 = range(len(sorted_missing_counts))
r2 = [x + bar_width for x in r1]

# Create a bar chart
plt.figure(figsize=(12, 8))
plt.bar(r1, sorted_missing_counts, color='blue', width=bar_width, edgecolor='grey', label='Missing Counts')
plt.bar(r2, sorted_all_counts, color='green', width=bar_width, edgecolor='grey', label='All Counts')

# Add titles and labels
plt.title('Comparison of Missing and All Date Counts')
plt.xlabel('Companies', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks([r + bar_width/2 for r in range(len(sorted_missing_counts))], labels, rotation=90)

# Add text labels for ratios on top of each bar
for i in range(len(sorted_missing_counts)):
    plt.text(r1[i], sorted_missing_counts[i] + 1, f'{sorted_ratios[i]:.2f}', ha='center', va='bottom', fontsize=10, color='blue')
    # plt.text(r2[i], sorted_all_counts[i] + 1, f'{sorted_ratios[i]:.2f}', ha='center', va='bottom', fontsize=10, color='green')

# Add legend
plt.legend()
plt.savefig("/data/kai/forecasting/eda_plots/missing_all_dates.png")
plt.show()


In [ ]:

raw_text_paths = sorted(glob("/data/kai/forecasting/summary/*_raw.txt"))


access_denied_counts = 0
total_counts = 0

for raw_path in tqdm(raw_text_paths, total=len(raw_text_paths)):
    raw_texts = load_text_from(raw_path)
    raw_texts = [r for r in raw_texts if r != "<SEP>" and r != ""]
    total_counts += len(raw_texts)
    access_denied_counts += sum(["access" in text.lower() and "denied" in text.lower() for text in raw_texts])
    # for i in trange(len(raw_texts)):
    #     output = pipe.run_llama(["I will provide a text that is a raw text data from the internet. Repond with a No if access to the page has been denied and there are not relevant stock information:", "```````" + raw_texts[i] + "```````"])
    #     access_answers.append(output)

    # break

In [ ]:
access_denied_counts,  total_counts

In [ ]:
len(raw_text_paths)

In [ ]:
pipe.run_llama(["", "what is your name in 10 sentences or less?"])

In [ ]:
raw_path = f"{data_dir}/{ticker}_{date_str}_raw.txt"

raw_texts = load_text_from(raw_path)
raw_texts = [r for r in raw_texts if r != "<SEP>" and r!= ""]

In [ ]:
raw_texts[2]

In [ ]:
summary_ending_prompt = f"Filter out irrelevant information and provide a concise summary including key numbers, growth trends, and the overall market outlook. Ensure to mention major stock movements, significant economic indicators, and any notable company-specific news. Do not make up false information. "
filter_prompt = "Keep the query the same, but please avoid any extraneous phrases or commentary such as 'Here is the filtered text' or 'I hope this helps.'"
summary_prompt = f"You are a helpful assistant that filters and summarizes stock news specifically for company with ticker symbol {ticker}."
combine_prompt = summary_prompt + " Combine the following summaries while preserving as much information as you can: "

In [ ]:
raw_texts[2]

In [ ]:
outputs = pipe.run_llama([summary_prompt, raw_texts[2]+summary_ending_prompt]*3, batch_size=3)

In [ ]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:  
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")